In [1]:
# get the paths to the code sorted out.  This is the wrong way to do it, and it needs sorting out as to the correct way
# Might need pip install developer mode so that a symlink is made to the databakersolo source code from the git checkout
# rather than it being copied deep into the site-packages directory tree
import sys
sys.path.insert(0, "/home/goatchurch/sensiblecode/databaker")
sys.path.insert(0, "/home/goatchurch/sensiblecode/xypath")
sys.path.insert(0, "/home/goatchurch/sensiblecode/messytables")
sys.path.insert(0, "/home/goatchurch/datalogging/anaconda3/lib/python3.5/site-packages/")  # for xlrd
sys.path.insert(0, "/home/goatchurch/datalogging/anaconda3/lib/python3.5/site-packages/xlutils-2.0.0-py3.5.egg/")  # for xlutils
import databaker
import databaker.constants
import xlutils




In [47]:
# The following is a block of code copied directly from an actual recipe without any changes

from databaker.constants import *

def per_file(tabs):    
    return PARAMS(0)

def per_tab(tab):           
     
    my_dict = {"6A_Top": 'CBAQ', "6A_Bottom": 'HCL3', "6D_Top": 'CBAS', "6D_Bottom":'HCL5',
               "7A_Top": 'CBAU', "7A_Bottom": 'HCK7', "7D_Top": 'CBAW', "7D_Bottom":'HCK9',
    }

    anchor = tab.filter(contains_string (my_dict[PARAMS(1)]))
    obs = anchor.shift(DOWN).expand(RIGHT).expand(DOWN).is_not_blank()

    # Getting all the junk out of the file
    unwanted = tab.excel_ref('AA1').expand(DOWN).expand(RIGHT)     
    unwanted = unwanted | anchor.expand(DOWN).filter(contains_string ('Area Analysis')).expand(RIGHT).expand(DOWN)
    unwanted = unwanted | tab.filter(contains_string ('indicates earliest revision')).expand(RIGHT).expand(DOWN)
    unwanted = unwanted | anchor.expand(DOWN).filter(contains_string ('Number')).shift(UP).expand(RIGHT).expand(DOWN)
    obs = obs - unwanted    
    
    # Dimension etc
    anchor.shift(0, -2).expand(RIGHT).is_not_blank().dimension("Area", CLOSEST, LEFT)
    anchor.shift(0, -1).expand(RIGHT).is_not_blank().dimension(MEASURETYPE, DIRECTLY, ABOVE)
    anchor.shift(-1, 0).expand(DOWN).is_not_blank().dimension(TIME, DIRECTLY, LEFT)        
    
    cat = tab.filter(contains_string (PARAMS(1)[0:2])).shift(RIGHT)
    cat.dimension("Mergers", CLOSEST, ABOVE)

    tab.dimension("Companies", "TempValue")
    
    yield obs



In [ ]:
# this is a set of parameters copied from a command in the batch file
sys.argv = ["Mergers_6_7_recipe.py", "--preview", "rftmatables_tcm77-415727.xls", "Table 6", "6A_Top"]


In [18]:
import databaker.databakersolo as ds
Opt = ds.Options()
ds.Globrecipeper_file = per_file
ds.Globrecipeper_tab = per_tab
databaker.utils.showtime_enabled = Opt.timing
databaker.constants.constant_params = Opt.params


In [27]:
# The following loop works (if there are multiple files), but here there is almost always only one, 
# so we are going to unroll it and inline the big DBSper_fileFUNC() function
#for fn in Opt.xls_files:
#    ds.DBSper_fileFUNC(fn, Opt)
spreadsheet = Opt.xls_files[0]


In [67]:
# Load in the table set and apply the given recipe.per_file() filter
tableset = xypath.loader.table_set(spreadsheet, extension='xls')
tabsfilter = per_file(tableset)   # function from the recipe
tabs = list(xypath.loader.get_sheets(tableset, tabsfilter))   
print("The list table names is", [ tab.name  for tab in tabs ])

# This concludes the business of the per_file() recipe function


The list table names is ['Table 6']


In [71]:
# Now we unroll the blocks of "yields" of segments given in the recipe.per_tab() function

conversionsegments = [ ]   # [ (tab, {int:dimension}, [obs]) ]
for tab in tabs:
    segmentgenerator = per_tab(tab)   # function from recipe
    if isinstance(segmentgenerator, xypath.xypath.Bag):
        segmentgenerator = [segmentgenerator]    # backward compatibility for return cases

    for segment in segmentgenerator:  # must be yielded so we can copy out tab.headers which are set between the function calls
        assert tab.headernames == [None]+[tab.headers[i].Dlabel  for i in range(1, max(tab.headers.keys())+1)]  # prove we can recreate the data
        conversionsegments.append((tab, tab.headers, segment))
        tab.headers, tab.max_header, tab.headernames = {}, 0, [None]  # reset the hidden list of dimensions/headers back to empty before the next block

# This concludes all the business of the per_tab() recipe function


got header Area: 103.117s,  3164.594s total
got header MEASURETYPE: 0.000s,  3164.595s total
got header TIME: 0.000s,  3164.595s total
got header Mergers: 0.003s,  3164.598s total
got header Companies: 0.000s,  3164.598s total


In [94]:
# Provide a quick summary of the conversions that have been scheduled
for conversionsegment in conversionsegments:
    tab, headers, segment = conversionsegment
    print("'%s':'%s' has %d cells of which %d have been selected" % (spreadsheet, tab.name, len(tab), len(segment)))
    for i, header in sorted(headers.items()):
        label = header.Dlabel
        if isinstance(label, int) and label < 0:
            label = databaker.constants.template.dimension_names[-label]
        print("    Dim:%d '%s' has %d headercells and is looked up in the %s direction" % (i, label, len(header.bag), str(header.direction)))


'rftmatables_tcm77-415727.xls':'Table 6' has 2176 cells of which 100 have been selected
    Dim:-6 'GEOG' has 10 headercells and is looked up in the (0, -1) direction
    Dim:-2 'STATUNIT' has 46 headercells and is looked up in the (-1, 0) direction
    Dim:1 'Area' has 5 headercells and is looked up in the (-1, 0) direction
    Dim:2 'Mergers' has 1 headercells and is looked up in the (0, -1) direction
    Dim:3 'Companies' has 2176 headercells and is looked up in the None direction


In [108]:
# This code block produces the preview spreadsheet if required.  
# We could intervene and plot the values of headers in an array to show how they are lining up
if Opt.preview:
    writer = xlutils.copy.copy(tableset.workbook)
    previewfilename = ds.filenames(spreadsheet, Opt)['preview']
    writer.save(previewfilename)
    for tab, headers, segment in conversionsegments:
        tableset = xypath.loader.table_set(previewfilename, extension='xls')   # load and save between each one
        writer = xlutils.copy.copy(tableset.workbook)
        ds.make_preview(writer, tab.index, headers, segment)
    print("Saving preview file '%s'" % previewfilename)
    writer.save(previewfilename)


Saving preview file 'preview-rftmatables_tcm77-415727-Mergers_6_7_recipe-Table 6,6A_Top.xls'


In [110]:
# We now do the conversions from the observations to the headers (This is the slow part)
batchrows = [ ]
for tab, headers, segment in conversionsegments:
    headernames = [None]+[headers[i].Dlabel  for i in range(1, max(headers)+1)]  # recreate value used in output conversion
    for ob_num, ob in enumerate(segment):
        values = ds.extract_dimension_values_for_ob(headers, ob, Opt.no_lookup_error)
        batchrows.append((values, headernames))


/home/goatchurch/sensiblecode/databaker/databaker/utils.py:47: UserWarning: Couldn't identify date '2010²'
  warnings.warn("Couldn't identify date {!r}".format(date))


In [119]:
# This is the final stage for outputting the bloated csv file
csv_file = ds.filenames(spreadsheet, Opt)['csv']
csv = ds.TechnicalCSV(csv_file, Opt.no_lookup_error)

# write the headers put out in the file
tab, headers, segment = conversionsegments[0]
headernames = [None]+[headers[i].Dlabel  for i in range(1, max(headers)+1)]  # recreate value used in output conversion
headerrow = csv.generate_header_row(headers, headernames)
csv.csv_writer.writerow(headerrow)  # note that only first batch of headernames is used

print("Outputting file '%s' with %d lines" % (csv_file, len(batchrows)))
print("The headerrow is:", headerrow)

# this spits out the actual rows
for values, headernames in batchrows:
    output_row = ds.yield_dimension_values(values, headernames)
    csv.output(output_row)
csv.footer()   # this closes the file

# And we're done

Outputting file 'data-rftmatables_tcm77-415727-Mergers_6_7_recipe-Table 6,6A_Top.csv' with 100 lines
The headerrow is: ['observation', 'data_marking', 'statistical_unit_eng', 'statistical_unit_cym', 'measure_type_eng', 'measure_type_cym', 'observation_type', 'empty', 'obs_type_value', 'unit_multiplier', 'unit_of_measure_eng', 'unit_of_measure_cym', 'confidentuality', 'empty1', 'geographic_area', 'empty2', 'empty3', 'time_dim_item_id', 'time_dim_item_label_eng', 'time_dim_item_label_cym', 'time_type', 'empty4', 'statistical_population_id', 'statistical_population_label_eng', 'statistical_population_label_cym', 'cdid', 'cdiddescrip', 'empty5', 'empty6', 'empty7', 'empty8', 'empty9', 'empty10', 'empty11', 'empty12', 'dim_id_1', 'dimension_label_eng_1', 'dimension_label_cym_1', 'dim_item_id_1', 'dimension_item_label_eng_1', 'dimension_item_label_cym_1', 'is_total_1', 'is_sub_total_1', 'dim_id_2', 'dimension_label_eng_2', 'dimension_label_cym_2', 'dim_item_id_2', 'dimension_item_label_eng_2